In [2]:
import pandas as pd

data = pd.read_csv('./data/orders.csv')
data = data[['Order Date', 'Sub-Category', 'Category', 'Sales', 'Quantity', 'Discount', 'Product ID']]
data['Order Date'] = pd.to_datetime(data['Order Date'])
data = data.sort_values(by='Order Date').reset_index(drop=True)
data['Price'] = data['Sales'] / data['Quantity'] * (1 - data['Discount'])
data['Quantity_lag_1'] = data.Quantity.shift(1)
data['Price_lag_1'] = data['Price'].shift(1)
data['Price_delta'] = data['Price'] / data.Price_lag_1
data

/var/folders/3q/37b0tth55xxdfn82hcm7ylp80000gn/T/ipykernel_59471/3488808202.py:5: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['Order Date'] = pd.to_datetime(data['Order Date'])


,Order Date,Sub-Category,Category,Sales,Quantity,Discount,Product ID,Price,Quantity_lag_1,Price_lag_1,Price_delta
0,2014-01-01,Storage,Office Supplies,66.120,4,0.000,OFF-ST-6230,16.530000,NaN,NaN,NaN
1,2014-01-01,Supplies,Office Supplies,120.366,3,0.100,OFF-SU-3002,36.109800,4.0,16.5300,2.184501
2,2014-01-01,Furnishings,Furniture,113.670,5,0.100,FUR-FU-4075,20.460600,3.0,36.1098,0.566622
3,2014-01-01,Storage,Office Supplies,408.300,2,0.000,OFF-ST-6261,204.150000,5.0,20.4606,9.977713
4,2014-01-01,Paper,Office Supplies,55.242,2,0.100,OFF-PA-3990,24.858900,2.0,204.1500,0.121768
...,...,...,...,...,...,...,...,...,...,...,...
51285,2017-12-31,Labels,Office Supplies,7.120,1,0.000,OFF-LA-4664,7.120000,3.0,45.1600,0.157662
51286,2017-12-31,Appliances,Office Supplies,209.300,2,0.000,OFF-AP-4222,104.650000,1.0,7.1200,14.698034
51287,2017-12-31,Fasteners,Office Supplies,3.024,3,0.200,OFF-FA-3330,0.806400,2.0,104.6500,0.007706
51288,2017-12-31,Accessories,Technology,276.600,4,0.000,TEC-AC-5200,69.150000,3.0,0.8064,85.751488


In [68]:
weekly_price_data = data[['Order Date', 'Price']].resample("W", on='Order Date').agg('mean').dropna()
weekly_price_data['lag_1'] = weekly_price_data.Price.shift(1)
weekly_price_delta = weekly_price_data.Price / weekly_price_data.lag_1
weekly_price_delta = pd.Series(weekly_price_delta.dropna().values)



weekly_quantities = data[['Order Date', 'Quantity']].resample("W", on='Order Date').agg('sum').dropna()
weekly_quantities['lag_1'] = weekly_price_data.Price.shift(1)
weekly_quantities = pd.Series(weekly_quantities.lag_1.dropna().values)


In [73]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error


weekly_quantity = data[['Order Date', 'Quantity_lag_1']].resample("W", on='Order Date').agg('sum').dropna()
time = pd.Series(list(range(208)))
X = pd.concat(
    (weekly_price_delta,
    weekly_quantities,
    time), axis=1
)
y = data[['Quantity', 'Order Date']].resample("W", on='Order Date').agg("sum").dropna().iloc[1:]


In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [75]:
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)
mean_absolute_percentage_error(y_test, y_pred)


0.30896444050628286

In [76]:
reg.coef_

array([[74.03703862,  4.23059578,  4.76997369]])